In [ ]:
import subprocess
import glob
import os

@tool
def find_pdf_files(directory: str) -> str:
    """
    Finds all PDF files in the specified directory and subdirectories.

    Args:
        directory: The directory path to search for PDF files

    Returns:
        A newline-separated list of PDF file paths
    """
    try:
        pdf_files = glob.glob(os.path.join(directory, "**/*.pdf"), recursive=True)
        if not pdf_files:
            return f"No PDF files found in {directory}"
        return "\n".join(pdf_files)
    except Exception as e:
        return f"Error finding PDF files: {str(e)}"


@tool
def convert_pdf_to_text(pdf_path: str, output_dir: str = "text_files") -> str:
    """
    Converts a PDF file to plain text using pdftotext.

    Args:
        pdf_path: Path to the PDF file
        output_dir: Directory to save the converted text file (default: text_files)

    Returns:
        Path to the converted text file or error message
    """
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Get the base filename without extension
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_path = os.path.join(output_dir, f"{base_name}.txt")

        # Run pdftotext command
        result = subprocess.run(
            ["pdftotext", "-layout", pdf_path, output_path],
            capture_output=True,
            text=True,
            check=True
        )

        return output_path
    except subprocess.CalledProcessError as e:
        return f"Error converting PDF: {e.stderr}"
    except FileNotFoundError:
        return "Error: pdftotext not found. Please install poppler-utils"
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def search_in_text(search_pattern: str, file_path: str, context_lines: int = 3) -> str:
    """
    Searches for a pattern in a text file using grep with context.

    Args:
        search_pattern: The text pattern to search for
        file_path: Path to the text file to search in
        context_lines: Number of lines of context to show around matches (default: 3)

    Returns:
        Search results with context or message if no matches found
    """
    try:
        result = subprocess.run(
            ["grep", "-i", "-C", str(context_lines), search_pattern, file_path],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            return result.stdout
        elif result.returncode == 1:
            return f"No matches found for '{search_pattern}' in {file_path}"
        else:
            return f"Error searching: {result.stderr}"
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def read_text_file(file_path: str, num_lines: Optional[int] = None) -> str:
    """
    Reads and returns the content of a text file.

    Args:
        file_path: Path to the text file
        num_lines: Optional number of lines to read from the beginning

    Returns:
        The file content or error message
    """
    try:
        if num_lines:
            result = subprocess.run(
                ["head", "-n", str(num_lines), file_path],
                capture_output=True,
                text=True,
                check=True
            )
            return result.stdout
        else:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

